In [1]:
from Udep2Mono.binarization import BinaryDependencyTree
from Udep2Mono.polarization import PolarizationPipeline
from Udep2Mono.util import btreeToList
from copy import deepcopy

2020-12-25 00:15:16 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | gum       |
| pos       | gum       |
| lemma     | gum       |
| depparse  | gum       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-12-25 00:15:16 INFO: Use device: cpu
2020-12-25 00:15:16 INFO: Loading: tokenize
2020-12-25 00:15:16 INFO: Loading: pos
2020-12-25 00:15:17 INFO: Loading: lemma
2020-12-25 00:15:17 INFO: Loading: depparse
2020-12-25 00:15:18 INFO: Loading: sentiment
2020-12-25 00:15:19 INFO: Loading: ner
2020-12-25 00:15:20 INFO: Done loading processors!


In [10]:
from pattern.en import pluralize, singularize

# TODO: Verb Phrase Patterns
# 1.Intransitive: subject + VI
# 2.Linking: subject + VL + NP/AdjP
# 3.Transitive: subject + VT + NP
# 4.Ditransitive: subject + VD + NP(indirect) + NP(direct)
# 5.Complex Transitive: subject + VC + NP(direct) + NP/AdjP
# 6.Open clausal complement: VB + to/that VP
class PhrasalGenerator:
    def __init__(self):
        self.deptree = None
        self.kb = {}
        self.treeLog = []
        self.polarLog = []
        self.mod_at_left = [
            "advmod", "amod", "advmod:count"]
        self.lexical_generation = {
             "advmod": self.generate_advmod,
             "advmod:count": self.generate_advmod,
             "amod": self.generate_amod,
             "acl": self.generate_acl_relcl,
             "acl:relcl": self.generate_acl_relcl,
             "advcl": self.generate_acl_relcl,
             "cc:preconj": self.generate_default,
             "det": self.generate_default,
             "det:predet": self.generate_default,
             "nsubj": self.generate_default,
             "nsubj:pass": self.generate_default,
        }
        '''"advmod": self.generate_advmod,
            "advmod:count": self.generate_advmod,
            "amod": self.generate_amod,
            "appos": self.generate_inherite,
            "aux": self.generate_aux,
            "aux:pass": self.generate_aux,
            "case": self.generate_case,
            "cc": self.generate_cc,
           
            "ccomp": self.generate_ccomp,
            "compound": self.generate_inherite,
            "compound:prt": self.generate_inherite,
            "conj": self.generate_inherite,
            "cop": self.generate_inherite,
            "csubj": self.generate_nsubj,
            "csubj:pass": self.generate_nsubj,
            "dep": self.generate_dep,
            
            "discourse": self.generate_discourse,
            "expl": self.generate_expl,
            "fixed": self.generate_inherite,
            "flat": self.generate_inherite,
            "goeswith": self.generate_inherite,
            "iobj": self.generate_inherite,
            "mark": self.generate_inherite,
            "nmod": self.generate_nmod,
            "nmod:npmod": self.generate_nmod,
            "nmod:tmod": self.generate_nmod,
            "nmod:poss": self.generate_nmod_poss,
            
            "nummod": self.generate_nummod,
            "obj": self.generate_obj,
            "obl": self.generate_obj,
            "obl:npmod": self.generate_oblnpmod,
            "obl:tmod": self.generate_inherite,
            "parataxis": self.generate_inherite,
            "xcomp": self.generate_obj,'''

    def deptree_generate(self, tree):
        self.treeLog = []
        self.deptree = tree
        self.generate(self.deptree)

    def generate(self, tree):
        if tree.val in self.lexical_generation.keys():
            self.lexical_generation[tree.val](tree)

    def delete_modifier(self, tree):
        tree.val = tree.right.val
        tree.mark = tree.right.mark
        tree.npos = tree.right.npos
        tree.id = tree.right.id
        tree.left = tree.right.left
        tree.right = tree.right.right

    def rollback(self, tree, backup):
        tree.val = backup.val
        tree.left = deepcopy(backup.left)
        tree.right = deepcopy(backup.right)
        tree.mark = backup.mark
        tree.npos = backup.npos
        tree.id = backup.id

    def generate_advmod(self, tree):
        "adv + VB | VB + adv => VB"
        left = tree.left
        right = tree.right
        backup = deepcopy(tree)

        if right.mark == "+":
            self.delete_modifier(tree)
            self.treeLog.append(self.save_tree())
            self.rollback(tree, backup)    

    def generate_amod(self, tree):  
        "amod + Noun => Noun"
        left = tree.left 
        right = tree.right
        backup = deepcopy(tree)

        if right.mark == "+":
            self.delete_modifier(tree)
            self.treeLog.append(self.save_tree())
            self.rollback(tree, backup)

    def generate_acl_relcl(self, tree):
        "Noun + relcl => Noun"
        left = tree.left
        right = tree.right
        backup = deepcopy(tree)

        if right.mark == "+":
            self.delete_modifier(tree)
            self.treeLog.append(self.save_tree())
            self.rollback(tree, backup)

    def generate_default(self, tree):
        left = tree.left
        right = tree.right 
        backup = deepcopy(tree)

        if right.npos is not None:
            if "NN" in right.npos and right.mark == "-":
                for adj in kb["ADJ"]:
                    amod_tree = self.buildTree(
                        {'rel': "amod",
                         'mod': adj,
                         'head': right.val,
                         'mark': "-",
                         'lid': right.id - (right.id-(right.id-1))/2,
                         'rid': right.id})
                    tree.right = amod_tree
                    self.treeLog.append(self.save_tree())
                for rel in kb["RCL"]:
                    amod_tree = self.buildTree(
                        {'rel': "acl:relcl",
                         'mod': rel,
                         'head': right.val,
                         'mark': "-",
                         'lid': right.id + ((right.id+1)-right.id)/2,
                         'rid': right.id})
                    tree.right = amod_tree
                    self.treeLog.append(self.save_tree())
                    self.rollback(tree, backup)
            elif "VB" in right.npos and right.mark == "-":
                for adj in kb["ADV"]:
                    amod_tree = self.buildTree(
                        {'rel': "advmod",
                         'mod': adj,
                         'head': right.val,
                         'mark': "-",
                         'lid': right.id - (right.id-(right.id-1))/2,
                         'rid': right.id})
                    tree.right = amod_tree
                    self.treeLog.append(self.save_tree())

        self.generate(left)
        self.generate(right)  
        
    def buildTree(self, config):
        left = BinaryDependencyTree(
            config['mod'], "N", "N", 1024, 
            wid=config['lid'], npos="JJ")
        right = BinaryDependencyTree(
            config['head'], "N", "N", 1024,
            wid=config['rid'], npos="NN")
        tree = BinaryDependencyTree(config['rel'], left, right, 1025)
        left.mark = config['mark']
        right.mark = config['mark']
        tree.mark = config['mark']
        return tree

In [14]:
sentences = ["Some red flowers need light", "All flowers need light"]
kb = {"ADJ": ["beautiful", "red", "fragret"], 
      "ADV": ["ergently", "clearly", "neccesaraly"]
      "RCL": ["which is beautiful", "which opens at night"]}

pipeline = PolarizationPipeline(sentences, verbose=1, parser="stanza")
pipeline.run_polarize_pipeline()
print("\nPolarization Complete")

phrasalGenerator = PhrasalGenerator()
for annotation in pipeline.annotations:
    print("================")
    print(annotation[0])
    phrasalGenerator.kb = kb
    phrasalGenerator.deptree_generate(annotation[4])
    for gen_tree in phrasalGenerator.treeLog:
        generated, queue, _, _ = btreeToList(gen_tree, len(annotation[1]), {}, 0)
        annotated = list(queue.popkeys())
        print(' '.join(annotated))
    

100%|██████████| 2/2 [00:00<00:00,  6.96it/s]
Polarization Complete
some↑ red↑ flowers↑ need↑ light↑



AttributeError: 'PhrasalGenerator' object has no attribute 'save_tree'